In [9]:
import nltk

nltk.download('punkt')

nltk.download('punkt_tab')

nltk.download('stopwords')

nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import cosine_similarity

from nltk.tokenize import word_tokenize, sent_tokenize

from nltk.corpus import stopwords

from nltk.tokenize import sent_tokenize

from nltk.stem import WordNetLemmatizer

import string
import random
import re

import streamlit as st

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [104]:
# chargement du fichier text
chemin_fichier = r"C:\Users\DELL\Desktop\chatboot\statistique.txt"
try:
    with open(chemin_fichier, 'r', encoding='utf-8') as fichier:
        contenu = fichier.read()
        #print(contenu)
except FileNotFoundError:
    print("Le fichier n'a pas été trouvé. Vérifiez le chemin et l'existence du fichier.")
except Exception as e:
    print(f"Une erreur est survenue : {e}")
def preprocess(text):
    # Convertir le texte en minuscules
    text = text.lower()
    
    # Supprimer les chiffres
    text = re.sub(r'\d+', '', text)
    
    # Supprimer la ponctuation
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenisation
    words = text.split()
    
    # Liste des mots vides (vous pouvez ajouter ou modifier cette liste)
    stop_words = set(stopwords.words('french'))  # Adapter la langue selon vos besoins
    stop_words.update(['à', 'de', 'et', 'la', 'le', 'les', 'des', 'un', 'une'])  # Mots vides supplémentaires
    
    # Filtrer les mots vides
    filtered_words = [word for word in words if word not in stop_words]
    
    # Rejoindre les mots filtrés en une seule chaîne
    processed_text = ' '.join(filtered_words)
    
    return processed_text

In [159]:
# Dictionnaire de questions avec plusieurs réponses possibles
responses = {
    "qu'est-ce que la fonction de répartition ?": [
        "Soit X une variable aléatoire, on appelle fonction de répartition de X, que l’on note F_X, la fonction définie sur R par : F_X(x) = P(X ≤ x)."
    ],
    "qu'est-ce qu'une quantile d'ordre q ?": [
        "On appelle quantile d’ordre q de la variable X, où q ∈ [0, 1], la valeur xq telle que P(X ≤ xq) = q ou, de même, FX(xq) = q."
    ],
    "qu'est-ce que la théorie des probabilités ?": [
        """La théorie des probabilités a pour objet l’étude des phénomènes aléatoires
        ou du moins considérés comme tels par l’observateur. Pour cela, on introduit le concept d’expérience aléatoire dont l’ensemble des résultats possibles
        constitue l’ensemble fondamental, noté habituellement Ω."""
    ],
    "qu'est-ce qu'une variable aléatoire ?": [
        """On parle de variable aléatoire (abréviation : v.a.) lorsque les résultats sont numériques, c’est-à-dire
        que Ω est identique à tout ou une partie de l’ensemble des nombres réels R. 
        On distingue habituellement :
        - les variables aléatoires discrètes pour lesquelles l’ensemble Ω des résultats possibles est un ensemble discret de valeurs numériques x1, x2, ..., xn, 
        fini ou infini (typiquement : l’ensemble des entiers naturels) ;
        - les variables aléatoires continues pour lesquelles l’ensemble Ω est tout R (ou un intervalle de R ou, plus rarement, une union d’intervalles)."""
    ],
    "qu'est-ce qu'une espérance mathématique ?": [
        """On appelle espérance mathématique de X, si elle existe, la valeur notée E(X) telle que :
        
        E(X) = Σ (xi * pX(xi)) dans le cas discret,
        
        E(X) = ∫ (x * fX(x) dx) dans le cas continu.
        
        Du point de vue du graphe de fX (respectivement pX), cette valeur représente le centre de gravité de la distribution."""
    ],
    "qu'est-ce que l'espérance d'une fonction ?": [
        """Soit g(X) une fonction de la v.a. X, alors :
        
        E(g(X)) = ∫ g(x) fX(x) dx dans le cas continu (si l’intégrale existe),
        
        E(g(X)) = Σ g(xi) pX(xi) dans le cas discret (si la somme existe).
        
        On voit donc que pour le calcul de E(g(X)), il suffit de remplacer la valeur de x par g(x) (ou xi par g(xi))."""
    ]
}


In [161]:
# Exemple de texte détaillé pour certaines questions
text_data = {
    "variable aléatoire": """On parle de variable aléatoire (abréviation : v.a.) lorsque les résultats sont numériques, 
    c'est-à-dire que chaque résultat d'une expérience aléatoire peut être associé à un nombre réel.""",
    
    "fonction de répartition": """Soit X une variable aléatoire, on appelle fonction de répartition de X, que l’on note F_X, 
    la fonction définie sur R par : F_X(x) = P(X ≤ x).""",
    
    "quantile d'ordre q": """On appelle quantile d’ordre q de la variable X, où q ∈ [0, 1], la valeur xq telle que 
    P(X ≤ xq) = q ou, de même, FX(xq) = q.""",
    
    "théorie des probabilités": """La théorie des probabilités est une branche des mathématiques qui étudie les phénomènes aléatoires 
    et peut être utilisée pour modéliser des situations d'incertitude.""",
    
    "espérance mathématique": """On appelle espérance mathématique de X, si elle existe, 
    la valeur notée E(X) telle que :
    
    E(X) = Σ (xi * pX(xi)) dans le cas discret,
    
    E(X) = ∫ (x * fX(x) dx) dans le cas continu.
    
    Du point de vue du graphe de fX (respectivement pX), cette valeur représente le centre de gravité de la distribution.""",
    
    "espérance d'une fonction": """Soit g(X) une fonction de la v.a X, alors :
    
    E(g(X)) = ∫ g(x) fX(x) dx dans le cas continu (si l’intégrale existe),
    
    E(g(X)) = Σ g(xi) pX(xi) dans le cas discret (si la somme existe).
    
    On voit donc que pour le calcul de E(g(X)), il suffit de remplacer la valeur de x par g(x) (ou xi par g(xi))."""
}


In [163]:
def normalize_query(query):
    normalized = query.lower().strip().translate(str.maketrans('', '', string.punctuation))
    return normalized

In [165]:
def get_most_relevant_sentence(user_query, text_data):
    sentences = ' '.join(text_data.values()).split('. ')
    sentences.append(user_query)
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(sentences)
    cosine_similarities = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    most_relevant_index = cosine_similarities.argmax()
    return sentences[most_relevant_index]

In [166]:
def chatbot(user_query):
    normalized_query = normalize_query(user_query)
    
    if "espérance mathématique" in normalized_query:
        return text_data["espérance mathématique"]
    elif "espérance" in normalized_query and "fonction" in normalized_query:
        return text_data["espérance d'une fonction"]
    elif "fonction de répartition" in normalized_query:
        return text_data["fonction de répartition"]
    elif "quantile d'ordre q" in normalized_query:
        return text_data["quantile d'ordre q"]
    elif "théorie des probabilités" in normalized_query:
        return text_data["théorie des probabilités"]
    elif "variable aléatoire" in normalized_query:  
        return text_data["variable aléatoire"]
    else:
        return get_most_relevant_sentence(user_query, text_data)


In [169]:
# Demander une question à l'utilisateur
#while True:
user_question = input("Posez votre question (ou tapez 'exit' pour quitter) : ")
    #if user_question.lower() == 'exit':
        #break
print(chatbot(user_question))

Posez votre question (ou tapez 'exit' pour quitter) :  qu'est-ce que l'espérance d'une fonction ?


Soit g(X) une fonction de la v.a X, alors :
    
    E(g(X)) = ∫ g(x) fX(x) dx dans le cas continu (si l’intégrale existe),
    
    E(g(X)) = Σ g(xi) pX(xi) dans le cas discret (si la somme existe).
    
    On voit donc que pour le calcul de E(g(X)), il suffit de remplacer la valeur de x par g(x) (ou xi par g(xi)).
